In [1]:
import pandas as pd
import numpy as np
import json
from datetime import date, timedelta
#from sodapy import Socrata
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import urllib.request
from selenium.webdriver.common.keys import Keys

In [2]:
date_2year_ago = (date.today() - timedelta(days=2*365))
date_today = date.today()
print('Las fechas a utilizar son: fecha dos anios atras: ' + str(date_2year_ago) + ' y fecha actual: ' + str(date_today))
# Cambiar esta variable si se necesita una fecha especifica
date_2year_ago = 20190000#int(date_2year_ago.strftime('%Y%m%d'))
date_today = int(date_today.strftime('%Y%m%d'))

Las fechas a utilizar son: fecha dos anios atras: 2019-04-25 y fecha actual: 2021-04-24


In [4]:
def first_df():
    # leemos desde la URL el json que contiene la tabla de informacion acerca de las tasas
    df = pd.read_json('https://www.datos.gov.co/resource/mcec-87by.json')
    # hacemos transformaciones a las fechas para tenerlas YYYYMMDD numericas
    df['vigenciadesde'] = pd.to_datetime(df['vigenciadesde']).dt.strftime('%Y%m%d').astype('int32')
    df['vigenciahasta'] = pd.to_datetime(df['vigenciahasta']).dt.strftime('%Y%m%d').astype('int32')
    # retornamos el data frame desde la fecha 2019
    return df[df['vigenciadesde'] >= date_2year_ago]

In [5]:
df  = first_df()
df

,valor,unidad,vigenciadesde,vigenciahasta
0,3640.07,COP,20210424,20210426
1,3630.81,COP,20210423,20210423
2,3639.12,COP,20210422,20210422
3,3636.26,COP,20210421,20210421
4,3606.42,COP,20210420,20210420
...,...,...,...,...
546,3128.07,COP,20190110,20190110
547,3164.75,COP,20190109,20190109
548,3208.56,COP,20190105,20190108
549,3241.20,COP,20190104,20190104


In [6]:
# Se agrupa por anio y mes. Se cuenta para saber cuantos registros hay por cada mes lo que equivale
# a la cantidad de dias por mes
tasa_spot = first_df()
periodo = (tasa_spot['vigenciadesde']/100).astype('int32')
data = periodo.to_list()
names = ['periodo']
new_df = pd.DataFrame(data, columns=names)
new_df.value_counts().sort_index()

periodo
201901     19
201902     19
201903     21
201904     19
201905     21
201906     19
201907     20
201908     21
201909     19
201910     22
201911     19
201912     20
202001     19
202002     20
202003     20
202004     20
202005     20
202006     18
202007     22
202008     19
202009     21
202010     22
202011     16
202012     21
202101     18
202102     19
202103     21
202104     16
dtype: int64

In [ ]:
# Enter the information from those sections here
#socrata_domain = 'datos.gov.co'
#socrata_dataset_identifier = 'mcec-87by'

# App Tokens can be generated by creating an account at https://opendata.socrata.com/signup
# Tokens are optional (`None` can be used instead), though requests will be rate limited.
#
# If you choose to use a token, run the following command on the terminal (or add it to your .bashrc)
# $ export SODAPY_APPTOKEN=<token>
#credentials = json.load(open('credentials.json'))
#socrata_token = credentials['TOKEN']


In [ ]:
#client = Socrata(socrata_domain, socrata_token)
#print("Domain: {domain:}\nSession: {session:}\nURI Prefix: {uri_prefix:}".format(**client.__dict__))

In [ ]:
#results = client.get(socrata_dataset_identifier)
#df = pd.DataFrame.from_dict(results)
#df_splt = df['vigenciadesde'].str.split('-')
#names = ['anio', 'mes', 'dia']
#data2 = df_splt.to_list()
#new_df2 = pd.DataFrame(data2, columns=names)
#new_df2.groupby(['anio', 'mes']).count()

In [ ]:
# Segundo dataframe
# Se configuran las opciones del navegador para que empiece maximixado y sin extensiones habilitadas
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

In [ ]:
# Funcion para eliminar duplicados de una lista
def remove_duplicate_list(lst):
    ''' Funcion para eliminar duplicados de una lista'''
    return list(dict.fromkeys(lst))

# Se crea instancia del driver que abrira el navegador web con las opciones definidas anteriormente
# Cambiar direccion a donde se encuentra el driver
driver_path = r'C:\Users\Efra\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(driver_path, options = options)
# Se le pasa el link a abrir en el navegador
driver.get('https://totoro.banrep.gov.co/analytics/saw.dll?Portal&PortalPath=%2Fshared%2FDashboards_T%2FD_Estad%C3%ADsticas%2FEstad%C3%ADsticas&NQUser=publico&NQPassword=publico123&lang=es&page=Tasas%20de%20inter%C3%A9s%20y%20sector%20financiero&pagina=Otras%20tasas%20de%20inter%C3%A9s')

WebDriverWait(driver, 15)
driver.switch_to.frame(driver.find_element_by_id('frame_dashboard'))

# Se da click en el boton otras tasas de interes
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.XPATH,
                                      '/html/body/div[7]/div/div/table/tbody/tr/td[1]/div/div/table/tbody/tr/td[2]/div/table/tbody/tr/td[2]/table/tbody/tr/td/div')))\
    .click()

# Se da click en el menu desplegable Indicador bancario de referencia IBR
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.CLASS_NAME,
                                      'sui-treeview-item-toggle-collapsed')))
driver.find_elements_by_class_name('sui-treeview-item-toggle-collapsed')[1].click()


#print(len(driver.find_elements_by_xpath("//a[@href]")))
# Se guardan todos los links asocaidos al atributo o etiqueta href
href = driver.find_elements_by_xpath("//a[@href]")
href_list = []
# Se recorre la lista href en busca de los links que pertenezcan a los insumos de IBR
for i in href:
    try:
        contains = str(i.get_attribute('href')).replace('%20', '').upper()
        #print(contains)
        # Si el href contiene alguno de los strings, es guardado en una lista que contendra los enlaces a descargar
        if('IBR_PLAZOOVERNIGHTNOMINALPARAUNRANGODEFECHASDADOIQY' in contains 
           or 'IBR_PLAZOUNMESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOTRESMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOMESVENCIDONOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOSEISMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           and 'DOWNLOAD' in contains):
            src = i.get_attribute('href')
            href_list.append(src)
            #print('saca algo ' + src)
    except:
        #print(error)
        print('error href')

# Como la lista posiblemente guarde enlaces duplicados, se aplica la funcion para remover duplicados y asi descargar una sola vez
# cada archivo
href_list = remove_duplicate_list(href_list)
print('Cantidad de archivos a descargar: ' + str(len(href_list)))
for j in href_list:
    try:
        driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 't') 
        driver.get(j)
        print('Archivo descargado asociado a: ' + j)
    except AssertionError as error:
        print(error)
        print('error link')
        
# Cambiar a 60 segundos para que descargue antes de cerrar
time.sleep(30)
driver.close()

# Tratamiento archivos IBR

In [26]:
def clean_ibr_df(ruta):
    '''Funcion que limpia y adecua los archivos IBR, como parametro recibe la ruta del archivo excel a tratar'''
    # Limpieza y adecuacion primer data frame IBR plazo overnight
    # Se lee la ruta donde se encuentra el archivo excel IBR y se le indica que los encabezados estaran en la posicion o celda 8
    # y que omita los pie de pagina
    df = pd.read_excel(ruta, header = 8, skipfooter = 7)
    # se renombran las columnas para mas claridad en el df
    df.rename(columns = {'Fecha (dd/mm/aaaa)' : 'Fecha', 'IBR' : 'IBR_efectivo', 'IBR.1' : 'IBR_nominal'}, inplace = True)
    # Se valida si la columna plazo existe, si no existe automaticamente le inserta una nueva columna al df y la rellena con el valor 0
    if 'Plazo real (días calendario)' not in df.columns:
        df['Plazo real (días calendario)'] = 0
    # Se toman solo esas 4 columnas del df
    df = df[['Fecha', 'Plazo real (días calendario)', 'IBR_efectivo', 'IBR_nominal']]
    # Se le hacen adecuaciones a la fecha para dejarla en formato YYYYMMDD numerico
    df['Fecha'] = pd.to_datetime(df['Fecha']).dt.strftime('%Y%m%d').astype('int32')
    # Se convierten las tasas de tipo texto a decimal
    df.IBR_efectivo.replace({',':'.'}, regex = True, inplace = True)
    df['IBR_efectivo'] = df['IBR_efectivo'].astype(float)
    df.IBR_nominal.replace({',':'.'}, regex = True, inplace = True)
    df['IBR_nominal'] = df['IBR_nominal'].astype(float)
    # Se rellena con 0 las tasas que esten en blanco o nulas
    df['IBR_efectivo'].fillna(0, inplace = True)
    df['IBR_nominal'].fillna(0, inplace = True)

    return df

In [25]:
def nominal_to_efectivo(tasa):
    '''Funcion para convertir tasa nominal a efectiva'''
    tasa = tasa / 100
    tasa_efectiva = (((1 + (tasa / 12))**12) - 1) * 100
    return tasa_efectiva

In [27]:
ibr_mes_nominal = clean_ibr_df(r'C:\Users\Efra\Downloads\1.4.IBR_Plazo mes vencido nominal para un rango de fechas dado IQY.xlsx')
ibr_mes_nominal

,Fecha,Plazo real (días calendario),IBR_efectivo,IBR_nominal
0,20120731,0,0.0,5.015
1,20120724,0,0.0,5.161
2,20120717,0,0.0,5.160
3,20120710,0,0.0,5.145
4,20120703,0,0.0,5.098
...,...,...,...,...
234,20080205,0,0.0,9.073
235,20080129,0,0.0,9.055
236,20080122,0,0.0,9.070
237,20080115,0,0.0,9.078


In [24]:
ibr_mes_nominal['IBR_efectivo'] = ibr_mes_nominal['IBR_nominal'].apply(nominal_to_efectivo)
ibr_mes_nominal

,Fecha,Plazo real (días calendario),IBR_efectivo,IBR_nominal
0,20120731,0,5.131893,5.015
1,20120724,0,5.284849,5.161
2,20120717,0,5.283800,5.160
3,20120710,0,5.268076,5.145
4,20120703,0,5.218822,5.098
...,...,...,...,...
234,20080205,0,9.459970,9.073
235,20080129,0,9.440416,9.055
236,20080122,0,9.456710,9.070
237,20080115,0,9.465402,9.078


In [31]:
ibr_overnight = clean_ibr_df(r'C:\Users\Efra\Downloads\1.1.IBR_Plazo overnight nominal para un rango de fechas dado IQY.xlsx')
ibr_overnight

,Fecha,Plazo real (días calendario),IBR_efectivo,IBR_nominal
0,20210422,1.0,1.747,1.708
1,20210421,1.0,1.742,1.703
2,20210420,1.0,1.739,1.700
3,20210419,1.0,1.739,1.700
4,20210416,3.0,1.731,1.693
...,...,...,...,...
3239,20080109,NaN,9.520,8.970
3240,20080108,NaN,9.509,8.960
3241,20080104,NaN,9.495,8.951
3242,20080103,NaN,9.520,8.970


In [ ]:
r = clean_ibr_df()
r.dtypes

In [ ]:
r.describe()
pd.set_option('display.min_rows', None)